In [2]:
%%bash

pip install --upgrade pip
pip install farm-haystack[colab,inference]


  Attempting uninstall: pip
    Found existing installation: pip 20.2.4
    Uninstalling pip-20.2.4:
      Successfully uninstalled pip-20.2.4
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.2/150.2 kB 793.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.7/48.7 kB 843.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 3.7 MB/s eta 0:00:00
  Using cached huggingface_hub-0.20.3-py3-none-any.whl.metadata (12 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 486.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 13.7 MB/s eta 0:00:00
  Using cached protobuf-4.25.2-cp37-abi3-macosx_10_9_universal2.whl.metadata (541 bytes)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 13.4 MB/s eta 0:00:00
  Using cached sniffio-1.3.0-py3-none-any.whl (10 kB)


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda 4.10.1 requires ruamel_yaml_conda>=0.11.14, which is not installed.


In [3]:
from haystack.telemetry import tutorial_running

tutorial_running(11)


/Users/simaochen/.julia/conda/3/lib/python3.8/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/Users/simaochen/.julia/conda/3/lib/python3.8/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/Users/simaochen/.julia/conda/3/lib/python3.8/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [4]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)


In [5]:
from haystack.document_stores import InMemoryDocumentStore

document_store = InMemoryDocumentStore(use_bm25=True)


INFO - haystack.modeling.utils -  Using devices: CPU - Number of GPUs: 0


In [6]:
# Optional step
import shutil

directory_path = '/content/data/text'
shutil.rmtree(directory_path)


FileNotFoundError: [Errno 2] No such file or directory: '/content/data/text'

In [7]:
# Optional step
import shutil

directory_path = '/content/data/text/__MACOSX'
shutil.rmtree(directory_path)


FileNotFoundError: [Errno 2] No such file or directory: '/content/data/text/__MACOSX'

In [ ]:
# from haystack.utils import fetch_archive_from_http, convert_files_to_docs, clean_wiki_text

# # Download and prepare data - 517 Wikipedia articles for Game of Thrones
# doc_dir = "data/tutorial11"
# s3_url = "https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/documents/wiki_gameofthrones_txt11.zip"
# fetch_archive_from_http(url=s3_url, output_dir=doc_dir)

# # convert files to dicts containing documents that can be indexed to our datastore
# got_docs = convert_files_to_docs(dir_path=doc_dir, clean_func=clean_wiki_text, split_paragraphs=True)
# document_store.delete_documents()
# document_store.write_documents(got_docs)


In [8]:
# from haystack.utils import fetch_archive_from_http

from google.colab import files

uploaded = files.upload()



ModuleNotFoundError: No module named 'google.colab'

In [41]:
import shutil
import os

zip_file_name = 'testingtexts.zip'
target_dir = '/content/data/text'
shutil.unpack_archive(zip_file_name, target_dir, 'zip')


In [42]:
from haystack.nodes import BM25Retriever, EmbeddingRetriever, FARMReader

bm25_retriever = BM25Retriever()

from haystack.pipelines import DocumentSearchPipeline
from haystack.utils import print_documents

p_retrieval = DocumentSearchPipeline(bm25_retriever)
# res = p_retrieval.run(query="Who is the father of Arya Stark?", params={"Retriever": {"top_k": 10}})
# print_documents(res, max_text_len=200)


In [43]:
from haystack.nodes import BM25Retriever, EmbeddingRetriever, FARMReader

# Initialize Sparse Retriever
bm25_retriever = BM25Retriever(document_store=document_store)

# Initialize embedding Retriever
embedding_retriever = EmbeddingRetriever(
    document_store=document_store, embedding_model="sentence-transformers/multi-qa-mpnet-base-dot-v1"
)
document_store.update_embeddings(embedding_retriever, update_existing_embeddings=False)

# Initialize Reader
reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2")


INFO:haystack.modeling.utils:Using devices: CPU - Number of GPUs: 0
INFO:haystack.nodes.retriever.dense:Init retriever using embeddings of model sentence-transformers/multi-qa-mpnet-base-dot-v1
INFO:haystack.document_stores.memory:Updating embeddings for 0 docs ...
Updating Embedding: 0 docs [00:00, ? docs/s]
INFO:haystack.modeling.utils:Using devices: CPU - Number of GPUs: 0
INFO:haystack.modeling.utils:Using devices: CPU - Number of GPUs: 0
INFO:haystack.modeling.model.language_model: * LOADING MODEL: 'deepset/roberta-base-squad2' (Roberta)
INFO:haystack.modeling.model.language_model:Auto-detected model language: english
INFO:haystack.modeling.model.language_model:Loaded 'deepset/roberta-base-squad2' (Roberta model) from model hub.
INFO:haystack.modeling.utils:Using devices: CPU - Number of GPUs: 0


In [46]:
from haystack.nodes import BM25Retriever

retriever = BM25Retriever(document_store=document_store)


In [47]:
from haystack.nodes import FARMReader

reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)


INFO:haystack.modeling.utils:Using devices: CPU - Number of GPUs: 0
INFO:haystack.modeling.utils:Using devices: CPU - Number of GPUs: 0
INFO:haystack.modeling.model.language_model: * LOADING MODEL: 'deepset/roberta-base-squad2' (Roberta)
INFO:haystack.modeling.model.language_model:Auto-detected model language: english
INFO:haystack.modeling.model.language_model:Loaded 'deepset/roberta-base-squad2' (Roberta model) from model hub.
INFO:haystack.modeling.utils:Using devices: CPU - Number of GPUs: 0


In [48]:
from haystack.pipelines import ExtractiveQAPipeline

pipe = ExtractiveQAPipeline(reader, bm25_retriever)


In [53]:
# prediction = pipe.run(
#     query="What is the name of dam?", params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}}
# )

import os
from haystack.pipelines.standard_pipelines import TextIndexingPipeline

files_to_index = [target_dir+ "/" + f for f in os.listdir(target_dir)]
indexing_pipeline = TextIndexingPipeline(document_store)
indexing_pipeline.run_batch(file_paths=files_to_index)


INFO:haystack.pipelines.base:It seems that an indexing Pipeline is run, so using the nodes' run method instead of run_batch.
Updating BM25 representation...: 100%|██████████| 11/11 [00:00<00:00, 5461.33 docs/s]


{'documents': [<Document: {'content': "INTRODUCTION\nThe Vajont reservoir disaster is a classic example of the consequences of the failure of engineers and geologists to understand the nature of the problem that they were trying to deal with. During the filling of the reservoir a block of approximately 270 million m3\ndetached from one wall and slid into the lake at velocities of up to 30 m sec-1 (approx. 110 km h-1). As a result a wave over topped the dam by 250 m and swept onto the valley below, with the loss of about 2500 lives. Remarkably the dam remained unbroken by the flood.\nLOCATION AND BACKGROUND\nVajont is located in the south-eastern part of the Dolomite Region of the Italian Alps, about 100 km north of Venice (Fig. 1). It was built as a part of the on-going, post-war development of Italy in order to provide HEP for the rapidly-expanding northern cities of Milan, Turin and Modena. Whilst a proposal to site a dam at this location was made in the 1920's, excavation of the sit

In [54]:
from haystack.pipelines import ExtractiveQAPipeline
from haystack.utils import print_answers

p_extractive_premade = ExtractiveQAPipeline(reader=reader, retriever=bm25_retriever)
res = p_extractive_premade.run(
    query="Who is the dam's name?", params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}}
)
print_answers(res, details="minimum")


Inferencing Samples: 100%|██████████| 1/1 [00:10<00:00, 10.66s/ Batches]

"Query: Who is the dam's name?"
'Answers:'
[   {   'answer': 'Hendron and Patten',
        'context': 'me that as such it represents a reactivation of an old '
                   'landslide (Hendron and Patten, 1985; Pasuto and Soldati, '
                   '1991), whilst others claim that it was'},
    {   'answer': 'Vajont reservoir disaster is a classic example of the '
                  'consequences of the failure of engineers and geologists to '
                  'understand the nature of the problem that they were trying '
                  'to deal with. During the filling of the reservoir a block '
                  'of approximately 270 million m3\n'
                  'detached from one wall and slid into the lake at velocities '
                  'of up to 30 m sec-1 (approx. 110 km h-1). As a result a '
                  'wave over topped the dam by 250 m and swept onto the valley '
                  'below, with the loss of about 2500 lives. Remarkably the '
              

In [55]:
from haystack.pipelines import DocumentSearchPipeline
from haystack.utils import print_documents

p_retrieval = DocumentSearchPipeline(embedding_retriever)
res = p_retrieval.run(query="What is the Dam's name?", params={"Retriever": {"top_k": 10}})
print_documents(res, max_text_len=200)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Query: What is the Dam's name?


In [56]:
from haystack.pipelines import Pipeline

# Custom built extractive QA pipeline
p_extractive = Pipeline()
p_extractive.add_node(component=bm25_retriever, name="Retriever", inputs=["Query"])
p_extractive.add_node(component=reader, name="Reader", inputs=["Retriever"])

# Now we can run it
res = p_extractive.run(query="What is the Dam's name?", params={"Retriever": {"top_k": 10}})
print_answers(res, details="minimum")

# Uncomment the following to generate the pipeline image
# p_extractive.draw("pipeline_extractive.png")


Inferencing Samples: 100%|██████████| 1/1 [00:10<00:00, 10.69s/ Batches]

"Query: What is the Dam's name?"
'Answers:'
[   {   'answer': 'Vajont',
        'context': 'INTRODUCTION\n'
                   'The Vajont reservoir disaster is a classic example of the '
                   'consequences of the failure of engineers and geologists to '
                   'understand the natur'},
    {   'answer': 'the worlds highest thin arch dam',
        'context': 'rved arch dam was, at 265.5 metres above the valley floor, '
                   'the worlds highest thin arch dam. The chord of the dam was '
                   '160 m, and the volume of impound'},
    {   'answer': 'Casso',
        'context': 've of water was pushed up the opposite bank and destroyed '
                   'the village of Casso, 260 m above lake level before '
                   'over-topping the dam by up to 245 m. The'},
    {   'answer': 'over-topping of the dam would be avoided.\n'
                  '\n'
                  '3. First Draw-Down of the Reservoir\n'
                  'Creep 

In [57]:
from haystack.nodes import JoinDocuments

# Create ensembled pipeline
p_ensemble = Pipeline()
p_ensemble.add_node(component=bm25_retriever, name="BM25Retriever", inputs=["Query"])
p_ensemble.add_node(component=embedding_retriever, name="EmbeddingRetriever", inputs=["Query"])
p_ensemble.add_node(
    component=JoinDocuments(join_mode="concatenate"), name="JoinResults", inputs=["BM25Retriever", "EmbeddingRetriever"]
)
p_ensemble.add_node(component=reader, name="Reader", inputs=["JoinResults"])

# Uncomment the following to generate the pipeline image
# p_ensemble.draw("pipeline_ensemble.png")

# Run pipeline
res = p_ensemble.run(
    query="What is the Dam's name?", params={"EmbeddingRetriever": {"top_k": 5}, "BM25Retriever": {"top_k": 5}}
)
print_answers(res, details="minimum")


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Inferencing Samples: 100%|██████████| 1/1 [00:04<00:00,  4.88s/ Batches]

"Query: What is the Dam's name?"
'Answers:'
[   {   'answer': 'Vajont',
        'context': 'INTRODUCTION\n'
                   'The Vajont reservoir disaster is a classic example of the '
                   'consequences of the failure of engineers and geologists to '
                   'understand the natur'},
    {   'answer': 'the worlds highest thin arch dam',
        'context': 'rved arch dam was, at 265.5 metres above the valley floor, '
                   'the worlds highest thin arch dam. The chord of the dam was '
                   '160 m, and the volume of impound'},
    {   'answer': 'Casso',
        'context': 've of water was pushed up the opposite bank and destroyed '
                   'the village of Casso, 260 m above lake level before '
                   'over-topping the dam by up to 245 m. The'},
    {   'answer': 'Hendron and Patten, 1985; Pasuto and Soldati, 1991), whilst '
                  'others claim that it was a first-time movement (Skempton, '
          

In [58]:
from haystack import BaseComponent
from typing import Optional, List


class CustomNode(BaseComponent):
    outgoing_edges = 1

    def run(self, query: str, my_optional_param: Optional[int]):
        # process the inputs
        output = {"my_output": ...}
        return output, "output_1"

    def run_batch(self, queries: List[str], my_optional_param: Optional[int]):
        # process the inputs
        output = {"my_output": ...}
        return output, "output_1"


In [59]:
class CustomQueryClassifier(BaseComponent):
    outgoing_edges = 2

    def run(self, query: str):
        if "?" in query:
            return {}, "output_2"
        else:
            return {}, "output_1"

    def run_batch(self, queries: List[str]):
        split = {"output_1": {"queries": []}, "output_2": {"queries": []}}
        for query in queries:
            if "?" in query:
                split["output_2"]["queries"].append(query)
            else:
                split["output_1"]["queries"].append(query)

        return split, "split"


# Here we build the pipeline
p_classifier = Pipeline()
p_classifier.add_node(component=CustomQueryClassifier(), name="QueryClassifier", inputs=["Query"])
p_classifier.add_node(component=bm25_retriever, name="BM25Retriever", inputs=["QueryClassifier.output_1"])
p_classifier.add_node(component=embedding_retriever, name="EmbeddingRetriever", inputs=["QueryClassifier.output_2"])
p_classifier.add_node(component=reader, name="QAReader", inputs=["BM25Retriever", "EmbeddingRetriever"])
# Uncomment the following to generate the pipeline image
# p_classifier.draw("pipeline_classifier.png")

# Run only the dense retriever on the full sentence query
res_1 = p_classifier.run(query="What is the Dam's name?")
print("Embedding Retriever Results" + "\n" + "=" * 15)
print_answers(res_1)

# Run only the sparse retriever on a keyword based query
res_2 = p_classifier.run(query="Dam's name")
print("BM25Retriever Results" + "\n" + "=" * 15)
print_answers(res_2)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Embedding Retriever Results
"Query: What is the Dam's name?"
'Answers:'
[]


Inferencing Samples: 100%|██████████| 1/1 [00:10<00:00, 10.64s/ Batches]

BM25Retriever Results
"Query: Dam's name"
'Answers:'
[   <Answer {'answer': 'Vajont', 'type': 'extractive', 'score': 0.26276788115501404, 'context': 'INTRODUCTION\nThe Vajont reservoir disaster is a classic example of the consequences of the failure of engineers and geologists to understand the natur', 'offsets_in_document': [{'start': 17, 'end': 23}], 'offsets_in_context': [{'start': 17, 'end': 23}], 'document_ids': ['32717dd3339bb101abc0d2b3c9a1aaa9'], 'meta': {'_split_id': 0}}>,
    <Answer {'answer': 'the worlds highest thin arch dam', 'type': 'extractive', 'score': 0.1586417257785797, 'context': 'rved arch dam was, at 265.5 metres above the valley floor, the worlds highest thin arch dam. The chord of the dam was 160 m, and the volume of impound', 'offsets_in_document': [{'start': 82, 'end': 114}], 'offsets_in_context': [{'start': 59, 'end': 91}], 'document_ids': ['5c0de1c932ca09dd015ab19a2a775dc9'], 'meta': {'_split_id': 1}}>,
    <Answer {'answer': 'Casso', 'type': 'extractive', 

In [60]:
# 1) You can set the `debug` attribute of a given node.
bm25_retriever.debug = True

# 2) You can provide `debug` as a parameter when running your pipeline
result = p_classifier.run(query="What is the Dam's name?", params={"BM25Retriever": {"debug": True}})

# 3) You can provide the `debug` paramter to all nodes in your pipeline
result = p_classifier.run(query="What is the Dam's name?", params={"debug": True})

result["_debug"]


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

{'Query': {'input': {'debug': True}, 'output': {}, 'exec_time_ms': 0.09},
 'QueryClassifier': {'input': {'query': "What is the Dam's name?",
   'debug': True},
  'output': {},
  'exec_time_ms': 0.11},
 'EmbeddingRetriever': {'input': {'root_node': 'Query',
   'query': "What is the Dam's name?",
   'debug': True},
  'output': {'documents': []},
  'exec_time_ms': 152.7},
 'QAReader': {'input': {'documents': [],
   'query': "What is the Dam's name?",
   'debug': True},
  'output': {'answers': []},
  'exec_time_ms': 0.3}}